In [3]:
from collections import defaultdict

import streamlit as st
import pickle as pkl

import torch
from sklearn.model_selection import train_test_split

from denoisers.ConditionalUnetDenoiser import ConditionalUnetDenoiser
from denoisers.ConditionalUnetMatrixDenoiser import ConditionalUnetMatrixDenoiser
from utils.pm_utils import discover_dk_process, remove_duplicates_dataset, pad_to_multiple_of_n
from utils.Config import Config
import plotly.express as px
import plotly.graph_objects as go
from dataset.dataset import SaladsDataset
from ddpm.ddpm_multinomial import Diffusion
from utils.initialization import initialize
import os
import argparse
import json
from streamlit.components.v1 import html
from pm4py.visualization.petri_net import visualizer as pn_visualizer
from pm4py.visualization.bpmn import visualizer as bpmn_visualizer
from pm4py import convert_to_bpmn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from utils.pm_utils import conformance_measure

In [4]:
with open("config.json", "r") as f:
    cfg_json = json.load(f)
    cfg = Config(**cfg_json)

In [5]:
with open(cfg.data_path, "rb") as f:
    dataset = pkl.load(f)

In [6]:
formatted_dataset = SaladsDataset(dataset['target'], dataset['stochastic'])

In [7]:
train_dataset, test_dataset = train_test_split(formatted_dataset, train_size=cfg.train_percent, shuffle=True,
                                               random_state=cfg.seed)

In [8]:
dk_process_model, dk_init_marking, dk_final_marking = discover_dk_process(train_dataset, cfg,
                                                                          preprocess=remove_duplicates_dataset)

In [28]:
import pm4py

nx = pm4py.convert_petri_net_to_networkx(dk_process_model, dk_init_marking, dk_final_marking)

In [32]:
import networkx as nwx

nwx.adjacency_matrix(nx).todense()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [21]:
from pm4py.objects.petri_net.utils import reachability_graph

rg = reachability_graph.construct_reachability_graph(dk_process_model, dk_init_marking)

ImportError: cannot import name 'create_networkx_undirected_graph' from 'pm4py.objects.petri_net.utils' (C:\Users\user\Anaconda3\envs\everything\Lib\site-packages\pm4py\objects\petri_net\utils\__init__.py)

In [13]:
rg.transitions

{(b65c9578-1d8c-4271-8642-bb28af4428a8, 'add_vinegar'),
 (ef885da4-71c8-4f82-a0dd-c9d1c7824cab, 'cut_cheese'),
 (4a8abb6e-95f8-4206-9bc5-b4e38f72fd29, 'cut_lettuce'),
 (init_loop_22, None),
 (b65c9578-1d8c-4271-8642-bb28af4428a8, 'add_vinegar'),
 (skip_16, None),
 (init_loop_17, None),
 (skip_3, None),
 (b65c9578-1d8c-4271-8642-bb28af4428a8, 'add_vinegar'),
 (cd50f9d7-07ac-4b83-aa9a-c5f4ce5a728b, 'place_cucumber_into_bowl'),
 (6bb26756-9d9e-4119-95b0-2578f71906cb, 'add_dressing'),
 (557ef8a3-999b-4ec2-bfa9-7ca26b2cb1a7, 'mix_dressing'),
 (ef885da4-71c8-4f82-a0dd-c9d1c7824cab, 'cut_cheese'),
 (ef885da4-71c8-4f82-a0dd-c9d1c7824cab, 'cut_cheese'),
 (ef885da4-71c8-4f82-a0dd-c9d1c7824cab, 'cut_cheese'),
 (skip_27, None),
 (83992b5f-6de5-462b-9795-edac7bf66226, 'mix_ingredients'),
 (28f639ff-2c4a-4534-a9ef-a7ece7ee8d76, 'add_oil'),
 (tauJoin_10, None),
 (skip_3, None),
 (b65c9578-1d8c-4271-8642-bb28af4428a8, 'add_vinegar'),
 (ef885da4-71c8-4f82-a0dd-c9d1c7824cab, 'cut_cheese'),
 (skip_26, No

In [16]:
transition_names = {tuple(s.strip(" '") for s in transition.name.strip("()").split(","))[1] for transition in
                    rg.transitions}

In [9]:
from utils.graph_utils import get_process_model_reachability_graph_transition_matrix

rg_nx, transition_matrix = get_process_model_reachability_graph_transition_matrix(dk_process_model, dk_init_marking)

In [8]:
from pm4py.objects.petri_net.utils import reachability_graph
import pm4py
import networkx as nx
import numpy as np
from scipy.sparse import coo_matrix

def get_process_model_reachability_graph_transition_matrix_sparse(process_model: pm4py.PetriNet, init_marking: pm4py.Marking):
    # Construct reachability graph
    rg = reachability_graph.construct_reachability_graph(process_model, init_marking)
    rg_nx = nx.DiGraph()

    for state in rg.states:
        rg_nx.add_node(state.name)

    # Extract unique transition names (using the second element in the tuple)
    transition_names = {
        tuple(s.strip(" '") for s in transition.name.strip("()").split(","))[1]
        for transition in rg.transitions
    }
    transition_name_index = {name: idx for idx, name in enumerate(sorted(transition_names))}

    # Add edges with labels to the graph
    for transition in rg.transitions:
        transition_name = tuple(s.strip(" '") for s in transition.name.strip("()").split(","))
        rg_nx.add_edge(
            transition.from_state.name,
            transition.to_state.name,
            label=transition_name
        )

    nodes = sorted(rg_nx.nodes())
    num_transitions = len(transition_names)
    num_nodes = len(nodes)

    # Instead of a dense (num_transitions, num_nodes, num_nodes) array,
    # build a sparse matrix (one per transition) by collecting the nonzero entries.
    # For efficiency, build a mapping from node name to its index.
    node_index = {node: idx for idx, node in enumerate(nodes)}

    # Create a dictionary to collect row, col, data for each transition index
    sparse_data = {i: {"rows": [], "cols": [], "data": []} for i in range(num_transitions)}

    for edge in tqdm(rg_nx.edges(data=True)):
        from_node = node_index[edge[0]]
        to_node = node_index[edge[1]]
        transition_name = edge[2]['label'][1]
        if transition_name in transition_name_index:
            trans_idx = transition_name_index[transition_name]
            sparse_data[trans_idx]["rows"].append(from_node)
            sparse_data[trans_idx]["cols"].append(to_node)
            sparse_data[trans_idx]["data"].append(1)
        else:
            raise RuntimeError(f"Encountered transition {transition_name} but it was not indexed.")

    # Build a list of sparse matrices (one per transition)
    sparse_transition_matrices = []
    for i in range(num_transitions):
        rows = sparse_data[i]["rows"]
        cols = sparse_data[i]["cols"]
        data = sparse_data[i]["data"]
        # Create a sparse matrix of shape (num_nodes, num_nodes)
        sp_mat = coo_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes), dtype=int)
        sparse_transition_matrices.append(sp_mat)

    return rg_nx, sparse_transition_matrices

In [9]:
rg_nx, sparse_transition_matrices = get_process_model_reachability_graph_transition_matrix(dk_process_model, dk_init_marking)

  0%|          | 0/3078969 [00:00<?, ?it/s]

In [10]:
import torch
from torch import nn
from scipy.sparse import coo_matrix

def convert_sparse_scipy_to_torch(sparse_list):
    """Converts a list of SciPy COO matrices to PyTorch sparse tensors."""
    torch_sparse_list = []
    for sp in sparse_list:
        sp = sp.tocoo()  # ensure COO format
        indices = torch.LongTensor([sp.row, sp.col])
        values = torch.tensor(sp.data, dtype=torch.float32)
        shape = sp.shape
        torch_sparse = torch.sparse_coo_tensor(indices, values, size=shape)
        torch_sparse_list.append(torch_sparse)
    return torch_sparse_list

In [11]:
transition_matrix = convert_sparse_scipy_to_torch(sparse_transition_matrices)

C:\Users\user\AppData\Local\Temp\ipykernel_36512\1406905325.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  indices = torch.LongTensor([sp.row, sp.col])


In [13]:
type(transition_matrix)

list